In [ ]:
#Some imports used later

import numpy as np
import scipy
from scipy import interpolate
from scipy.interpolate import interp1d

from solver import *

import matplotlib.pyplot as plt
from matplotlib import ticker, cm
from numpy import ma
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import seaborn as sns

plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')
plt.rc('font', size=15)
plt.rc('figure', autolayout=True)
plt.rc('axes', titlesize=18, labelsize=18)
plt.rc('lines', linewidth=2, markersize=6)
plt.rc('legend', fontsize=15)
plt.rc('figure')

%matplotlib notebook
#%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

In [ ]:
###--- Fiducial params

N_lst = np.linspace(-15., 10., int(1e5))# The time grid

As = 2.12605*1e-9
ns = 0.966
Omega_M = 0.32
Omega_R = 1e-5
H_not = 1.258*1e-61 #72 km/Mpc/s in inverse Planck time units

phi_ini = -10.
initial_condition = [phi_ini, 0.] 
N_star = 2./(1. - ns)

In [ ]:
sln_lst_model_1 = []
sln_lst_model_2 = []
###############-------------##############################-------------###############
alpha_lst = [7./3., 6./3., 5./3., 4./3., 3./3., 2./3., 1./3.]

for alpha_indx, alpha_tmp in enumerate(alpha_lst):
    M = np.sqrt(144.*np.pi**2*alpha_tmp*N_star*As/(2.*N_star - 3.*alpha_tmp)**3)# In Planck mass units
    M = M/H_not
    
    gamma_scan = True
    h_scan = False
    params = ["EXP_model_1", gamma_scan, h_scan, M, 0., alpha_tmp, Omega_M, Omega_R]
    sln_tmp = solve(N_lst, initial_condition, params)
    
    sln_lst_model_1.append(sln_tmp)
    
    gamma_scan = True
    h_scan = False
    params = ["EXP_model_2", gamma_scan, h_scan, M, 0., alpha_tmp, Omega_M, Omega_R]
    sln_tmp = solve(N_lst, initial_condition, params)
    
    sln_lst_model_2.append(sln_tmp)

params = ["LCDM", True, False, M, 0., alpha_tmp, Omega_M, Omega_R]
sln_LCDM = solve(N_lst, initial_condition, params)


In [ ]:
###--- Just to see if we got adequate results 
params_LCDM = sln_LCDM[0]

Omega_L = params_LCDM[1]**2*np.exp(-2.*params_LCDM[2])/3.

print(Omega_L, 1. - Omega_M - Omega_R)

In [ ]:
###--- We start producing the figures
colorparams = alpha_lst

normalize = mcolors.Normalize(vmin=np.min(colorparams), vmax=np.max(colorparams))
colormap = ListedColormap(sns.cubehelix_palette(len(colorparams), reverse = False).as_hex())
#cm.viridis_r#

In [ ]:
###--- wDE figure

fig = plt.figure()#figsize = (12.5, 6.)
axes = fig.subplots(2, 1, sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

for indx, sln_tmp in enumerate(sln_lst_model_1):
    
    color = colormap(normalize(alpha_lst[indx]))
    phi_prime = sln_tmp[1][:,1]
    eps_phi = phi_prime**2/2.
    axes[0].plot(N_lst, -1. + 2.*eps_phi/3., c = color, label = r"$\alpha = $" + str(alpha_lst[indx]))
    
for indx, sln_tmp in enumerate(sln_lst_model_2):
    
    color = colormap(normalize(alpha_lst[indx]))
    phi_prime = sln_tmp[1][:,1]
    eps_phi = phi_prime**2/2.
    axes[1].plot(N_lst, -1. + 2.*eps_phi/3., c = color, label = r"$\alpha = $" + str(alpha_lst[indx]))
    axes[1].axhline([-1. + 2./9./alpha_lst[indx]], c = color, ls = "--", lw = 1.)

s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
s_map.set_array(colorparams)

halfdist = (colorparams[0] - colorparams[1])/2.0
boundaries = np.linspace(colorparams[::-1][0] - halfdist, colorparams[::-1][-1] + halfdist, len(colorparams) + 1)

cbar = fig.colorbar(s_map, spacing='proportional', boundaries=boundaries, \
                    ticks=np.array(alpha_lst), ax=axes.flat, pad = 0.03, location = 'top') 
cbar.ax.set_xticklabels(['7/3', '6/3', '5/3', '4/3', '3/3', '2/3', '1/3'])
cbarlabel = r'$\alpha$'
cbar.set_label(cbarlabel, fontsize=20)

axes[0].text(-2.5, -0.93, "Exp-model I",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)

axes[1].text(-2.5, -0.4, "Exp-model II",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)

axes[0].set_xlim([-5., 10.])
axes[1].set_xlim([-5., 10.])

axes[0].axvline([0.], c = "gray", ls = "--", zorder = 1)
axes[0].axhline([-1.], c = "gray", ls = "--", zorder = 1)

axes[1].axvline([0.], c = "gray", ls = "--", zorder = 1)
axes[1].axhline([-1.], c = "gray", ls = "--", zorder = 1)

axes[0].set_ylabel(r"$w_\mathrm{DE}$")
axes[1].set_ylabel(r"$w_\mathrm{DE}$")
axes[1].set_xlabel(r"$N$")

axes[0].yaxis.set_minor_locator(MultipleLocator(1e-2))
#axes[1].yaxis.set_minor_locator(MultipleLocator(1e-1))

plt.savefig("Figs/wDE.pdf", bbox_inches='tight')

In [ ]:
###--- h figure
a_lst = np.exp(N_lst)
z_lst = 1./a_lst - 1.

sol_Hubble_E_sqr_LCDM = sln_LCDM[3]

fig = plt.figure()#figsize = (12.5, 6.)
axes = fig.subplots(2, 1, sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

for indx, sln_tmp in enumerate(sln_lst_model_1):
    
    color = colormap(normalize(alpha_lst[indx]))
    sol_Hubble_E_sqr = sln_tmp[3]
    axes[0].plot(z_lst, np.sqrt(sol_Hubble_E_sqr/sol_Hubble_E_sqr_LCDM), c = color)

for indx, sln_tmp in enumerate(sln_lst_model_2):
    
    color = colormap(normalize(alpha_lst[indx]))
    sol_Hubble_E_sqr = sln_tmp[3]
    axes[1].plot(z_lst, np.sqrt(sol_Hubble_E_sqr/sol_Hubble_E_sqr_LCDM), \
                 c = color, label = r"$\alpha = $" + str(alpha_lst[indx]))

s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
s_map.set_array(colorparams)

halfdist = (colorparams[0] - colorparams[1])/2.0

boundaries = np.linspace(colorparams[::-1][0] - halfdist, colorparams[::-1][-1] + halfdist, len(colorparams) + 1)

cbar = fig.colorbar(s_map, spacing='proportional', boundaries=boundaries, \
                    ticks=np.array(alpha_lst), ax=axes.flat, pad = 0.03, location = 'top') 
cbar.ax.set_xticklabels(['7/3', '6/3', '5/3', '4/3', '3/3', '2/3', '1/3'])
cbarlabel = r'$\alpha$'
cbar.set_label(cbarlabel, fontsize=20)

axes[0].set_xlim([0., 1.5])
axes[1].set_xlim([0., 1.5])
axes[0].set_ylim([0.995, 1.015])
axes[1].set_ylim([0.99, 1.08])


axes[0].axhline([1.], c = "gray", ls = "--", zorder = 1)
axes[1].axhline([1.], c = "gray", ls = "--", zorder = 1)

axes[0].set_ylabel(r"$H/H_{\Lambda\mathrm{CDM}}$")
axes[1].set_ylabel(r"$H/H_{\Lambda\mathrm{CDM}}$")
axes[1].set_xlabel(r"$z$")

axes[0].set_yscale('linear')


axes[0].text(1.2, 1.013, "Exp-model I",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)
axes[1].text(1.2, 1.07, "Exp-model II",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)

axes[0].yaxis.set_minor_locator(MultipleLocator(1e-3))
axes[1].yaxis.set_minor_locator(MultipleLocator(1e-2))

plt.savefig("Figs/h.pdf", bbox_inches='tight')

In [ ]:
###--- f figure
N_lst_f_evolve = np.linspace(-4., 0., int(1e4))

a_lst = np.exp(N_lst_f_evolve)
z_lst = 1./a_lst - 1.

sol_Hubble_E_sqr_LCDM = sln_LCDM[3]
f_LCDM = f_solve(N_lst_f_evolve, N_lst, sol_Hubble_E_sqr_LCDM, Omega_M)

fig = plt.figure()#figsize = (12.5, 6.)
axes = fig.subplots(2, 1, sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

for indx, sln_tmp in enumerate(sln_lst_model_1):
    color = colormap(normalize(alpha_lst[indx]))
    
    sol_Hubble_E_sqr = sln_tmp[3]
    f = f_solve(N_lst_f_evolve, N_lst, sol_Hubble_E_sqr, Omega_M)

    axes[0].plot(z_lst, f/f_LCDM, c = color, label = r"$\alpha = $" + str(alpha_lst[indx]))
  

for indx, sln_tmp in enumerate(sln_lst_model_2):
    color = colormap(normalize(alpha_lst[indx]))
    
    sol_Hubble_E_sqr = sln_tmp[3]
    f = f_solve(N_lst_f_evolve, N_lst, sol_Hubble_E_sqr, Omega_M)

    axes[1].plot(z_lst, f/f_LCDM, c = color, lw = 1.5, label = r"$\alpha = $" + str(alpha_lst[indx]))

s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
s_map.set_array(colorparams)

halfdist = (colorparams[0] - colorparams[1])/2.0
boundaries = np.linspace(colorparams[::-1][0] - halfdist, colorparams[::-1][-1] + halfdist, len(colorparams) + 1)


cbar = fig.colorbar(s_map, spacing='proportional', boundaries=boundaries, \
                    ticks=np.array(alpha_lst), ax=axes.flat, pad = 0.03, location = 'top')
cbar.ax.set_xticklabels(['7/3', '6/3', '5/3', '4/3', '3/3', '2/3', '1/3'])
cbarlabel = r'$\alpha$'
cbar.set_label(cbarlabel, fontsize=20)

axes[0].set_xlim([0., 1.5])
axes[1].set_xlim([0., 1.5])
axes[0].set_ylim([0.985, 1.002])
axes[1].set_ylim([0.92, 1.01])


axes[0].axhline([1.], c = "gray", ls = "--", zorder = 1)
axes[1].axhline([1.], c = "gray", ls = "--", zorder = 1)

axes[0].text(1.25, 0.99, "Exp-model I",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)
axes[1].text(1.25, 0.95, "Exp-model II",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)

axes[0].yaxis.set_minor_locator(MultipleLocator(1e-3))
axes[1].yaxis.set_minor_locator(MultipleLocator(1e-2))

axes[0].set_ylabel(r"$f/f_{\Lambda\mathrm{CDM}}$")
axes[1].set_ylabel(r"$f/f_{\Lambda\mathrm{CDM}}$")
axes[1].set_xlabel(r"$z$")

plt.savefig("Figs/f.pdf", bbox_inches='tight')

In [ ]:
###--- The money plot

fig = plt.figure(figsize = (7, 4.9))
ax = fig.gca()

sigma_ns_Model_2 = 1e-4
ns_Model_2 = 0.966
ns_standard = 0.96

ns_lst = np.linspace(0.9599, 0.967, 10000)

ax.plot(ns_lst, np.exp( - (ns_lst - ns_Model_2)**2 / (2. * sigma_ns_Model_2**2) ),
         lw = 2, color='black', label = "GC+WL, SKA2, Exp-model II")
ptx = np.linspace(ns_Model_2 - sigma_ns_Model_2, ns_Model_2 + sigma_ns_Model_2, 1000)
pty = scipy.stats.norm.pdf(ptx, ns_Model_2, sigma_ns_Model_2)/\
                scipy.stats.norm.pdf([ns_Model_2], ns_Model_2, sigma_ns_Model_2)
plt.fill_between(ptx, pty, color='black', alpha='0.2')
ptx = np.linspace(ns_Model_2 - 2.*sigma_ns_Model_2, ns_Model_2 + 2.*sigma_ns_Model_2, 1000)
pty = scipy.stats.norm.pdf(ptx, ns_Model_2, sigma_ns_Model_2)/\
                scipy.stats.norm.pdf([ns_Model_2], ns_Model_2, sigma_ns_Model_2)
plt.fill_between(ptx, pty, color='black', alpha='0.2')
ptx = np.linspace(ns_Model_2 - 3.*sigma_ns_Model_2, ns_Model_2 + 3.*sigma_ns_Model_2, 1000)
pty = scipy.stats.norm.pdf(ptx, ns_Model_2, sigma_ns_Model_2)/\
                scipy.stats.norm.pdf([ns_Model_2], ns_Model_2, sigma_ns_Model_2)
plt.fill_between(ptx, pty, color='black', alpha='0.2')



#######################################################################################################
colorparams_N_star = [57., 57.5, 58., 58.5, 59., 59.5, 60., 60.5]

normalize_N_star = mcolors.Normalize(vmin=np.min(colorparams_N_star), vmax=np.max(colorparams_N_star))
colormap_N_star = cm.viridis_r

ax.axvline([1. - 2./50.], ls = "--", lw = 3, \
               color = "black", \
               label = "Non-quintessential," + r"$\;N_\ast = $" + str(50))


ax.axvline([-10.], ls = "--", lw = 3, \
               color = "white", \
               label = " ")
ax.axvline([-10.], ls = "--", lw = 3, \
               color = "white", \
               label = "Quintessential")

for N_star_tmp in colorparams_N_star:
    
    ax.axvline([1. - 2./N_star_tmp], ls = "-", lw = 3, \
               color = colormap_N_star(normalize_N_star(N_star_tmp)), \
               label = r"$\;\;\;\;N_\ast = $" + str(N_star_tmp), alpha = 0.85, zorder = 2)
    



ax.set_xlabel(r"$n_\mathrm{s}$")
ax.set_ylabel(r"$P/P_\mathrm{max}$")

ax.set_ylim([0., 1.05])
ax.set_xlim([min(ns_lst), max(ns_lst)])

ax.legend(loc = (0.05, 0.02))

plt.savefig("Figs/ns.pdf", bbox_inches='tight')

In [ ]:
###--- wDE figure

fig = plt.figure()#figsize = (12.5, 6.)
axes = fig.subplots(2, 1, sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

for indx, sln_tmp in enumerate(sln_lst_model_1):
    
    color = colormap(normalize(alpha_lst[indx]))
    eps = sln_tmp[2]
    
    axes[0].plot(N_lst, -1. + 2.*eps/3., c = color, label = r"$\alpha = $" + str(alpha_lst[indx]))
    
for indx, sln_tmp in enumerate(sln_lst_model_2):
    
    color = colormap(normalize(alpha_lst[indx]))
    eps = sln_tmp[2]
    
    axes[1].plot(N_lst, -1. + 2.*eps/3., c = color, label = r"$\alpha = $" + str(alpha_lst[indx]))
    axes[1].axhline([-1. + 2./9./alpha_lst[indx]], c = color, ls = "--", lw = 1.)

s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
s_map.set_array(colorparams)

halfdist = (colorparams[0] - colorparams[1])/2.0
boundaries = np.linspace(colorparams[::-1][0] - halfdist, colorparams[::-1][-1] + halfdist, len(colorparams) + 1)

cbar = fig.colorbar(s_map, spacing='proportional', boundaries=boundaries, \
                    ticks=np.array(alpha_lst), ax=axes.flat, pad = 0.03, location = 'top') 
cbar.ax.set_xticklabels(['7/3', '6/3', '5/3', '4/3', '3/3', '2/3', '1/3'])
cbarlabel = r'$\alpha$'
cbar.set_label(cbarlabel, fontsize=20)

axes[0].text(4.5, 0., "Exp-model I",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)

axes[1].text(4.5, 0., "Exp-model II",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)

axes[0].set_xlim([-15., 10.])
axes[1].set_xlim([-15., 10.])

axes[0].axvline([0.], c = "gray", ls = "--", zorder = 1)
axes[0].axhline([-1.], c = "gray", ls = "--", zorder = 1)

axes[1].axvline([0.], c = "gray", ls = "--", zorder = 1)
axes[1].axhline([-1.], c = "gray", ls = "--", zorder = 1)

axes[0].set_ylabel(r"$w_\mathrm{eff}$")
axes[1].set_ylabel(r"$w_\mathrm{eff}$")
axes[1].set_xlabel(r"$N$")

plt.savefig("Figs/w_eff.pdf", bbox_inches='tight')

In [ ]:
###--- phi evolution figure

fig = plt.figure()#figsize = (12.5, 6.)
axes = fig.subplots(2, 1, sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

for indx, sln_tmp in enumerate(sln_lst_model_1):
    
    color = colormap(normalize(alpha_lst[indx]))
    phi = sln_tmp[1][:,0]

    axes[0].plot(N_lst, phi, c = color, label = r"$\alpha = $" + str(alpha_lst[indx]))
    
for indx, sln_tmp in enumerate(sln_lst_model_2):
    
    color = colormap(normalize(alpha_lst[indx]))
    phi = sln_tmp[1][:,0]

    axes[1].plot(N_lst, phi, c = color, label = r"$\alpha = $" + str(alpha_lst[indx]))

s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
s_map.set_array(colorparams)

halfdist = (colorparams[0] - colorparams[1])/2.0
boundaries = np.linspace(colorparams[::-1][0] - halfdist, colorparams[::-1][-1] + halfdist, len(colorparams) + 1)

cbar = fig.colorbar(s_map, spacing='proportional', boundaries=boundaries, \
                    ticks=np.array(alpha_lst), ax=axes.flat, pad = 0.03, location = 'top') 
cbar.ax.set_xticklabels(['7/3', '6/3', '5/3', '4/3', '3/3', '2/3', '1/3'])
cbarlabel = r'$\alpha$'
cbar.set_label(cbarlabel, fontsize=20)

axes[0].text(-10., -11., "Exp-model I",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)

axes[1].text(-10., -15., "Exp-model II",
         rotation = 0,
         horizontalalignment='center',
         verticalalignment='top',
         multialignment='center',
         color = "k",
         bbox=dict(facecolor="gray", alpha = .2, boxstyle='round,pad=0.2'), zorder = 2, size=18)

axes[0].set_xlim([-15., 10.])
axes[1].set_xlim([-15., 10.])

axes[0].axvline([0.], c = "gray", ls = "--", zorder = 1)
axes[0].axhline([-10.], c = "gray", ls = "--", zorder = 1)

axes[1].axvline([0.], c = "gray", ls = "--", zorder = 1)
axes[1].axhline([-10.], c = "gray", ls = "--", zorder = 1)

axes[0].set_ylabel(r"$\varphi$")
axes[1].set_ylabel(r"$\varphi$")
axes[1].set_xlabel(r"$N$")


plt.savefig("Figs/phi.pdf", bbox_inches='tight')

In [ ]:
###--- Small tests

alpha_tmp = 7./3.
As_fid = 2.12605*1e-9
ns_fid = 0.966


############################
###--- Fiducial run
ns = ns_fid
As = As_fid

N_star = 2./(1. - ns)
M = np.sqrt(144.*np.pi**2*alpha_tmp*N_star*As/(2.*N_star - 3.*alpha_tmp)**3)# In Planck mass units
M = M/H_not

gamma_scan = True
h_scan = False
params = ["EXP_model_2", gamma_scan, h_scan, M, 0., alpha_tmp, Omega_M, Omega_R]
sln_fid = solve(N_lst, initial_condition, params)

gamma_fid = sln_fid[0][2]

############################
###--- 2 percent change in gamma

ns = ns_fid
As = As_fid

gamma_scan = False
h_scan = True
N_star = 2./(1. - ns)
M = np.sqrt(144.*np.pi**2*alpha_tmp*N_star*As/(2.*N_star - 3.*alpha_tmp)**3)# In Planck mass units
M = M/H_not

params = ["EXP_model_2", gamma_scan, h_scan, M, 0.98*gamma_fid, alpha_tmp, Omega_M, Omega_R]
sln_change_gamma = solve(N_lst, initial_condition, params)

############################
###--- 5 percent change in n_s

ns = 0.95*ns_fid
As = As_fid

gamma_scan = False
h_scan = True
N_star = 2./(1. - ns)
M = np.sqrt(144.*np.pi**2*alpha_tmp*N_star*As/(2.*N_star - 3.*alpha_tmp)**3)# In Planck mass units
M = M/H_not


params = ["EXP_model_2", gamma_scan, h_scan, M, gamma_fid, alpha_tmp, Omega_M, Omega_R]
sln_change_ns = solve(N_lst, initial_condition, params)

############################


In [ ]:
N_lst_f_evolve = np.linspace(-4., 0., int(1e4))

a_lst = np.exp(N_lst_f_evolve)
z_lst = 1./a_lst - 1.

sol_Hubble_E_sqr = sln_fid[3]
f_fid = f_solve(N_lst_f_evolve, N_lst, sol_Hubble_E_sqr, Omega_M)

sol_Hubble_E_sqr = sln_change_gamma[3]
f_change_gamma = f_solve(N_lst_f_evolve, N_lst, sol_Hubble_E_sqr, Omega_M)

sol_Hubble_E_sqr = sln_change_ns[3]
f_change_ns = f_solve(N_lst_f_evolve, N_lst, sol_Hubble_E_sqr, Omega_M)

In [ ]:
fig = plt.figure()
ax = fig.gca()

plt.plot(z_lst, f_change_gamma/f_fid, c = "k", label = "2 percent change in " + r"$\gamma$")
plt.plot(z_lst, f_change_ns/f_fid, c = "darkred", label = "5 percent change in " + r"$n_\mathrm{s}$")


ax.set_xlim([0., 2.])
ax.set_xlabel("z")
ax.set_ylabel(r"$f/f_\mathrm{fid}$")

ax.legend()

plt.savefig("Figs/test.pdf", bbox_inches='tight')